In [2]:
import pandas as pd
import os
import numpy as np
from loguru import logger
import sys
# Load dateloader
from scipy.stats import gaussian_kde
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
import contact_state_classification as csc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
experiment_dir = csc.config.path["experiment_dir"]
cs_classifier = csc.CSClassifier(experiment_dir=experiment_dir, dataset_name=csc.config.path["dataset_name"])

cs_classifier.csd_data_dict.keys()

cs_classifier.predict(input_data=np.ones([1, 12]))


2021-11-13 15:05:52.908 | INFO     | contact_state_classification.cs_classifier:get_dataset_information:79 - All classes from the dataset RoboticsProject2510_2 are ['CS1' 'CS2' 'CS3' 'CS5' 'CS6']: 


ValueError: X has 12 features, but KNeighborsClassifier is expecting 36 features as input.

In [ ]:
cs1_idx_list = [15, 16, 17, 18, 19]
cs3_idx_list = [5, 6, 7, 8, 9]
cs5_idx_list = [10, 11, 12, 13, 14]
cs6_idx_list = [0, 1, 2, 3, 4]

def show_plots(col):
    cs1_df = cs_classifier.csd_data_df.loc[cs1_idx_list][col]
    cs3_df = cs_classifier.csd_data_df.loc[cs3_idx_list][col]
    cs5_df = cs_classifier.csd_data_df.loc[cs5_idx_list][col]
    cs6_df = cs_classifier.csd_data_df.loc[cs6_idx_list][col]

    # cs1_df_theta = pd.DataFrame([*cs1_df], columns = ['act' + str(x) for x in range(1, 13)])

    df_cs1 =  pd.DataFrame({'CS1' : [item for item in cs1_idx_list for i in range(12)],
                            'action' : list(range(1, 13)) * len(cs1_idx_list),
                            col : pd.DataFrame([*cs1_df]).to_numpy().flatten()})

    df_cs3 =  pd.DataFrame({'CS3' : [item for item in cs3_idx_list for i in range(12)],
                            'action' : list(range(1, 13)) * len(cs3_idx_list),
                            col : pd.DataFrame([*cs3_df]).to_numpy().flatten()})

    df_cs5 =  pd.DataFrame({'CS5' : [item for item in cs5_idx_list for i in range(12)],
                            'action' : list(range(1, 13)) * len(cs5_idx_list),
                            col : pd.DataFrame([*cs5_df]).to_numpy().flatten()})

    df_cs6 =  pd.DataFrame({'CS6' : [item for item in cs6_idx_list for i in range(12)],
                            'action' : list(range(1, 13)) * len(cs6_idx_list),
                            col : pd.DataFrame([*cs6_df]).to_numpy().flatten()})

    sns_plot = sns.relplot(x="action", y=col, hue="CS1",
                dashes=False, markers=True, kind="line", data=df_cs1)
    sns_plot.savefig("cs1.png")

    sns_plot = sns.relplot(x="action", y=col, hue="CS3",
                dashes=False, markers=True, kind="line", data=df_cs3)
    sns_plot.savefig("cs3.png")

    sns_plot = sns.relplot(x="action", y=col, hue="CS5",
                dashes=False, markers=True, kind="line", data=df_cs5)
    sns_plot.savefig("cs5.png")

    sns_plot = sns.relplot(x="action", y=col, hue="CS6",
                dashes=False, markers=True, kind="line", data=df_cs6)
    sns_plot.savefig("cs6.png")


In [ ]:
# import hiplot as hip
#
# hip.Experiment.from_dataframe(cs1_df_theta).display()


In [ ]:
# show_plots("d_ee_theta")
#

In [ ]:
# show_plots("d_ee_phi")

In [ ]:
show_plots("obs_ee_theta")

In [ ]:
# show_plots("obs_ee_phi")

In [ ]:
# show_plots("dist")

